In [1]:
import pandas as pd
import basedosdados as bd

In [2]:
query1 = """
SELECT DISTINCT c.ano, c.id_candidato_bd, c.nome_urna, c.sigla_partido, c.cargo, 
               c.genero, c.raca, rc.resultado, SUM(rc.votos) votos
       
FROM basedosdados.br_tse_eleicoes.candidatos c
INNER JOIN basedosdados.br_tse_eleicoes.resultados_candidato rc
ON c.id_candidato_bd = rc.id_candidato_bd
AND c.ano = rc.ano
WHERE c.sigla_uf = 'RJ'
AND c.cargo in ('deputado estadual', 'deputado federal', 'vereador')
AND c.ano in (2022, 2020, 2018, 2016, 2014, 2012, 2010, 2008)
GROUP BY c.ano, c.id_candidato_bd, c.nome_urna, c.sigla_partido, c.cargo, 
         c.genero, c.raca, rc.resultado
"""

query2 = """
SELECT DISTINCT c.ano, c.id_candidato_bd, SUM(bc.valor_item) bens_declarados

FROM basedosdados.br_tse_eleicoes.candidatos c
INNER JOIN basedosdados.br_tse_eleicoes.bens_candidato bc
ON c.id_candidato_bd = bc.id_candidato_bd
AND c.ano = bc.ano
WHERE c.sigla_uf = 'RJ'
AND c.cargo in ('deputado estadual', 'deputado federal', 'vereador')
AND c.ano in (2022, 2020, 2018, 2016, 2014, 2012, 2010, 2008)
GROUP BY c.ano, c.id_candidato_bd
"""

query3 = """
SELECT DISTINCT c.ano, c.id_candidato_bd, SUM(dc.valor_despesa) despesas_campanha

FROM basedosdados.br_tse_eleicoes.candidatos c
INNER JOIN basedosdados.br_tse_eleicoes.despesas_candidato dc
ON c.id_candidato_bd = dc.id_candidato_bd
AND c.ano = dc.ano
WHERE c.sigla_uf = 'RJ'
AND c.cargo in ('deputado estadual', 'deputado federal', 'vereador')
AND c.ano in (2022, 2020, 2018, 2016, 2014, 2012, 2010, 2008)
GROUP BY c.ano, c.id_candidato_bd
"""

dataCandidatos = bd.read_sql(query1, billing_project_id="analise-eleitoral-330723")
dataBens = bd.read_sql(query2, billing_project_id="analise-eleitoral-330723")
dataDespesas = bd.read_sql(query3, billing_project_id="analise-eleitoral-330723")

Downloading: 100%|██████████| 62540/62540 [00:05<00:00, 12418.85rows/s]


In [3]:
data0 = dataCandidatos.merge(dataBens, on=['ano', 'id_candidato_bd'], how='left')
data1 = data0.merge(dataDespesas, on=['ano', 'id_candidato_bd'], how='left')

In [4]:
data2 = data1.sort_values(by=['ano', 'votos'], ascending=False)
data2.to_csv('datasets/Data.csv', index=False)

In [5]:
query4 = """
SELECT DISTINCT c.ano, c.id_municipio, c.id_candidato_bd, c.nome_urna, c.sigla_partido, c.cargo, 
               c.genero, c.raca, rc.resultado, SUM(rc.votos) votos
       
FROM basedosdados.br_tse_eleicoes.candidatos c
INNER JOIN basedosdados.br_tse_eleicoes.resultados_candidato rc
ON c.id_candidato_bd = rc.id_candidato_bd
AND c.ano = rc.ano
WHERE c.sigla_uf = 'RJ'
AND c.cargo in ('vereador')
AND c.ano in (2020, 2016, 2012, 2008)
GROUP BY c.ano, c.id_candidato_bd, c.nome_urna, c.sigla_partido, c.cargo, 
         c.genero, c.raca, rc.resultado, c.id_municipio
"""

query5 = """
SELECT DISTINCT id_municipio, geometria 
FROM basedosdados.br_geobr_mapas.municipio
WHERE sigla_uf = 'RJ'
"""

dataVereadores = bd.read_sql(query4, billing_project_id="analise-eleitoral-330723")
geometrias = bd.read_sql(query5, billing_project_id="analise-eleitoral-330723")

Downloading: 100%|██████████| 92/92 [00:01<00:00, 67.23rows/s]


In [6]:
Vereadores = dataVereadores.merge(geometrias, on=['id_municipio'], how='left')
Vereadores_genero = Vereadores.groupby(['ano', 'genero', 'raca', 'resultado', 'id_municipio', 'geometria'])['genero'].count().reset_index(name = 'n')
Vereadores_raca = Vereadores.groupby(['ano', 'genero', 'raca', 'resultado', 'id_municipio', 'geometria'])['raca'].count().reset_index(name = 'n')

In [7]:
Vereadores_genero.to_csv('datasets/VereadoresGenero.csv', index=False)
Vereadores_raca.to_csv('datasets/VereadoresRaca.csv', index=False)